In [24]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import json


In [25]:
import hashlib

bipartite = {}


In [26]:
pairs = {}

for month in [
              '01', '02', '03', '04', '05', '06', 
              '07', '08', '09', '10', '11', '12'
             ]:
    fusion = []
    for k in range(1, 5):
        data = pd.read_csv('/PLEASE_DOWNLOAD_THE_DATA_FROM_s/patterns_backfill/2020/05/07/12/2019/%s/patterns-part%d.csv.gz' % (month, k))
        fusion.append(data)
        
    fusion = pd.concat(fusion)
    data = fusion
    rr = list(data['visitor_home_cbgs'])
    pois = list(data['safegraph_place_id'])

    for iii in tqdm(range(len(rr))):
        r = rr[iii]
        poi = pois[iii]
        
        js = json.loads(r)
        js = {int(k): js[k] for k in js}
        
        cbgs = list(set([int(key) for key in js]))
        cbgs = [cbg for cbg in cbgs]
        
        for j in range(len(cbgs)):
            
            cj = cbgs[j]
            if cj not in pairs:
                pairs[cj] = {}

            if poi not in pairs[cj]:
                pairs[cj][poi] = 0

            pairs[cj][poi] += js[cbgs[j]]


100%|██████████| 4207034/4207034 [03:47<00:00, 18524.56it/s]


In [27]:
place_info = [pd.read_csv('/data/social/places/myLocalDirectory/2020/03/core_poi-part1.csv.gz'),
              pd.read_csv('/data/social/places/myLocalDirectory/2020/03/core_poi-part2.csv.gz'),
              pd.read_csv('/data/social/places/myLocalDirectory/2020/03/core_poi-part3.csv.gz'),
              pd.read_csv('/data/social/places/myLocalDirectory/2020/03/core_poi-part4.csv.gz'),
              pd.read_csv('/data/social/places/myLocalDirectory/2020/03/core_poi-part5.csv.gz')
             ]

place_info = pd.concat(place_info)

In [28]:
poi2top_category = {}
poi2sub_category = {}

for i, poi_row in place_info.iterrows():
    poi = poi_row['safegraph_place_id']
    top_category = poi_row['top_category']
    sub_category = poi_row['sub_category']
    poi2top_category[poi] = top_category
    poi2sub_category[poi] = sub_category

In [29]:
place_info.head()

,safegraph_place_id,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,iso_country_code,phone_number,open_hours,category_tags
0,sg:001341fe7e794ab6bd65bb80759a1ac6,NaN,Old Firehouse School,NaN,NaN,Child Day Care Services,Child Day Care Services,624410.0,37.890704,-122.118315,984 Moraga Rd,Lafayette,CA,94549,US,1.925284e+10,"{ ""Mon"": [[""8:00"", ""17:30""]], ""Tue"": [[""8:00"",...",NaN
1,sg:002921847f104c93a6df47bb831dedd2,NaN,Tenth Avenue Park,NaN,NaN,"Museums, Historical Sites, and Similar Institu...",Nature Parks and Other Similar Institutions,712190.0,42.255103,-89.085241,825 10th St,Rockford,IL,61104,US,NaN,NaN,NaN
2,sg:002d2e9f047d47ca8661abb8939096cf,NaN,Hunter Village Wines & Liquors,NaN,NaN,"Beer, Wine, and Liquor Stores","Beer, Wine, and Liquor Stores",445310.0,42.214633,-74.220827,7261 Rt 23a,Hunter,NY,12442,US,1.518263e+10,"{ ""Mon"": [], ""Tue"": [[""13:00"", ""18:00""]], ""Wed...",NaN
3,sg:0060616e5f184585a026a0b7e8192fbd,NaN,Fat Moe's,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,40.948824,-74.154553,204 Diamond Bridge Ave,Hawthorne,NJ,7506,US,1.973949e+10,NaN,"Mediterranean Food,Smoothie & Juice Bar,Sandwi..."
4,sg:0061c1d8e124406cbcc97e40eb1dd585,NaN,Jungle Rapids Family Fun Park,NaN,NaN,Amusement Parks and Arcades,Amusement and Theme Parks,713110.0,34.207249,-77.863010,5320 Oleander Dr,Wilmington,NC,28403,US,1.910791e+10,NaN,NaN


In [15]:
np.save('poi2top_category.npy', poi2top_category)
np.save('poi2sub_category.npy', poi2sub_category)


In [30]:
pois = set([poi for cbg in pairs for poi in pairs[cbg]])
pois = pois - set(place_info['parent_safegraph_place_id'].dropna())

In [31]:
new_pairs = {}

for cbg in pairs:
    d = {poi: pairs[cbg][poi] for poi in pairs[cbg] if poi in pois}
    if d:
        new_pairs[cbg] = d

In [33]:
np.save('pairs_full_12.npy', new_pairs)